In [3]:
#pip install transformers torch (in cmd)


[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
import pandas as pd
from transformers import pipeline


In [16]:
data_path = "../data/processed/cleaned_mental_health.csv"
df = pd.read_csv(data_path)
df.head()


,Unnamed: 0,statement,status,cleaned_statement
0,0,oh my gosh,Anxiety,oh gosh
1,1,"trouble sleeping, confused mind, restless hear...",Anxiety,trouble sleeping confused mind restless heart ...
2,2,"All wrong, back off dear, forward doubt. Stay ...",Anxiety,wrong back dear forward doubt stay restless re...
3,3,I've shifted my focus to something else but I'...,Anxiety,ive shifted focus something else im still worried
4,4,"I'm restless and restless, it's been a month n...",Anxiety,im restless restless month boy mean


In [19]:
df['status'].value_counts()


status
Normal                  16213
Depression              15403
Suicidal                10648
Anxiety                  3841
Bipolar                  2777
Stress                   2587
Personality disorder     1077
Name: count, dtype: int64

In [23]:
MAX_SAMPLES_PER_CLASS = 3000

def capped_sampling(group):
    if len(group) > MAX_SAMPLES_PER_CLASS:
        return group.sample(n=MAX_SAMPLES_PER_CLASS, random_state=42)
    else:
        return group  # take all if less than cap

df_capped = (
    df.groupby("status", group_keys=False)
      .apply(capped_sampling)
)

df_capped.shape


C:\Users\areba\AppData\Local\Temp\ipykernel_16780\608057066.py:11: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(capped_sampling)


(18441, 4)

In [24]:
df_capped['status'].value_counts()


status
Anxiety                 3000
Depression              3000
Normal                  3000
Suicidal                3000
Bipolar                 2777
Stress                  2587
Personality disorder    1077
Name: count, dtype: int64

In [25]:
emotion_classifier = pipeline(
    "text-classification",
    model="j-hartmann/emotion-english-distilroberta-base",
    top_k=1,
    truncation=True,
    max_length=512
)


Device set to use cpu


In [26]:
sample_text = df_capped['cleaned_statement'].iloc[0]
emotion_classifier(sample_text)


[[{'label': 'fear', 'score': 0.9904913306236267}]]

In [27]:
texts = df_capped['cleaned_statement'].tolist()

emotion_outputs = emotion_classifier(
    texts,
    batch_size=8,   # safe for CPU laptops
    truncation=True
)

df_capped['emotion'] = [result[0]['label'] for result in emotion_outputs]


In [28]:
df_capped[['cleaned_statement', 'emotion']].head()


,cleaned_statement,emotion
33782,health anxiety checking temperature time day c...,fear
52543,way sleep better cant sleep night med didnt help,sadness
51923,lamictal anyone good review lamictal,neutral
51483,anyone tried emdr therapy started thought like...,surprise
34940,would may ecg picked worried yesterday drillin...,fear


In [29]:
df_capped['emotion'].value_counts()


emotion
sadness     6932
fear        4989
anger       1886
neutral     1851
joy         1603
surprise    1095
disgust       85
Name: count, dtype: int64

In [30]:
output_path = "../data/processed/emotion_mental_health.csv"
df_capped.to_csv(output_path, index=False)


In [31]:
df_capped.shape


(18441, 5)

In [32]:
df_capped['status'].value_counts()


status
Anxiety                 3000
Depression              3000
Normal                  3000
Suicidal                3000
Bipolar                 2777
Stress                  2587
Personality disorder    1077
Name: count, dtype: int64